<a href="https://colab.research.google.com/github/carlosarroyave/AutoEnc-GAN/blob/main/AutoEnc%26GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython import display

In [ ]:
#plot original images vs reconstructed images

def plot_mnist_autoencoder(x, xpred, cmap='gray', vmin=0, vmax=1):
    # Crear una figura y ejes para mostrar las imágenes originales y reconstruidas
    fig, ax = plt.subplots(2, x.shape[0], figsize=(20, 5))

    # Iterar sobre cada imagen en x y xpred para mostrarlas en subplots correspondientes
    for i, class_ in enumerate(range(x.shape[0])):
        # Mostrar la imagen original en la fila 0 del subplot
        ax[0, i].imshow(x[i], cmap=cmap, vmin=vmin, vmax=vmax)
        ax[0, i].set_xticks([])
        ax[0, i].set_yticks([])

        # Mostrar la imagen reconstruida en la fila 1 del subplot
        ax[1, i].imshow(xpred[i], cmap=cmap, vmin=vmin, vmax=vmax)
        ax[1, i].set_xticks([])
        ax[1, i].set_yticks([])

    # Mostrar la figura completa con todas las subplots
    plt.show()

# Ejemplo de uso de la función
# Suponiendo que tienes imágenes en las variables x e xpred
# plot_mnist_autoencoder(x, xpred)

Identificación de por PCA de los resultados de cada capa del modelo utilizado

In [ ]:
def plot_car_PCA(z, y, dim=2):
    """
    Esta función realiza PCA (Análisis de Componentes Principales) en los datos de z y
    luego visualiza las proyecciones en un gráfico de scatter para cada capa L en z.

    Args:
    - z: Objeto que contiene los datos a analizar (debe tener atributos L1, L2, ..., Ln).
    - y: Etiquetas de clase para colorear los puntos en el gráfico de dispersión.
    - dim: Número de dimensiones para la proyección PCA (por defecto es 2).

    """
    num_L = z.L_n
    fig, axes = plt.subplots(1, num_L, figsize=(10*num_L, 10))

    for i in range(num_L):
        # Obtener los datos de la capa L correspondiente
        name_Z = f"L{i+1}"
        L = getattr(z, name_Z).numpy()
        shp = L.shape

        # Realizar PCA dependiendo de la forma de los datos en la capa L
        if len(shp) == 2:
            zpca = PCA(n_components=dim).fit_transform(L.reshape((L.shape[0], L.shape[1])))
        elif len(shp) == 3:
            zpca = PCA(n_components=dim).fit_transform(L.reshape((L.shape[0], L.shape[1]*L.shape[2])))
        elif len(shp) == 4:
            zpca = PCA(n_components=dim).fit_transform(L.reshape((L.shape[0], L.shape[1]*L.shape[2]*L.shape[3])))

        # Normalizar los datos de PCA entre 0 y 1
        zpca = (zpca - zpca.min()) / (zpca.max() - zpca.min())

        # Graficar la proyección PCA en un gráfico de scatter
        axes[i].scatter(zpca[:, 0], zpca[:, 1], c=y, s=10, cmap="tab10")
        axes[i].set_title(f"L{i+1}")

    plt.subplots_adjust(wspace=0.5)
    plt.legend()
    plt.show()

# Ejemplo de uso de la función
# plot_car_PCA(z, y, dim=2)

## DataSet utlizado de keras fashion_mnist

El dataset esta calsificado en las siguientes clases :  # T-shirt/top, 	Trouser, Pullover,	Dress, 	Coat, Sandal, Shirt, 	Sneaker, Bag, Ankle boot

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

División de los datos en Train, test y valid y se normalizan los datos de 0 a 1

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [ ]:
X_train[0].shape

In [ ]:
plt.imshow(X_train[0], cmap="gray", vmin=0, vmax=1)
plt.axis('off')
plt.show()

Se mezclan los datos para garantizar que el modelo no tenga overfitting por el orden de los datos en el entrenamiento.

In [ ]:
tamano = len(X_train)
indice_mezcla = np.random.permutation(tamano)
x_tr = X_train[indice_mezcla].reshape([tamano,28,28,1])
tamano = len(X_valid)
indice_mezcla = np.random.permutation(tamano)
x_vl = X_valid[indice_mezcla].reshape([tamano,28,28,1])
y_vl = y_valid[indice_mezcla]
tamano = len(X_test)
indice_mezcla = np.random.permutation(tamano)
x_ts = X_test[indice_mezcla].reshape([tamano,28,28,1])

# Modelo Autoencoder con Regularizadores

Se crea un modelo de autoencoder con una entrada de imagen (28,28) buscando reconstruir la imagen de entrada.

Se utliza la siguiente arquitectura para el encoder y decoder.

![Arquitectura](Imagenes/ArquitecturaAE.png)

y se utliza la API de los modelos que permite keras para poder seleccionar que tipo de regularización se va a implementar.

### Ruido gausiano:
El primer regularizador es ruido gausiano en la capa de entrada, generando un grado de aleatoriedad a los datos de entrada y evitar el sobre entrenamiento.

### Dropout:
Se genera un apagado de las neuronas de forma aleatoria en la capa implementada lo que genera que algunas neuronas no sean las que se especialicen en el entrenamiento y se pueda distribuir el aprendizaje en todas las neuronas de la capa.

###  Regularización L1 Lasso:
Introduce un termino a la función de costo (L) añadiendo al valor el absoluto de los pesos de la capa, generando que los pesos sean pequeños y no se genere una neurona con mayor importancia que las otras neuronas de la capa.

$$
L_1(X,\omega ) =  L(X,\omega ) + \lambda \sum \left |\omega_i  \right |
$$

###  Regularización L2 Ridge:

Introduce un termino a la función de costo (L) añadiendo a su valor la suma de los cuadrados de los parametros ($\omega$) y se introduce un parametro ($\lambda$) para ajustar el peso de estos cuadrados de los pesos.

$$
L_2(X,\omega ) =  L(X,\omega ) + \lambda \sum \omega_i^{2}
$$